In [18]:
import re
import pandas as pd
import numpy as np
import MySQLdb
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, KFold
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Dropout, LSTM, Merge, Input, Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.utils import np_utils
from sklearn.externals import joblib
from sqlalchemy import create_engine

In [19]:
def binary(Y):
    Y[np.where(Y > 0)] = 1
    Y[np.where(Y <= 0)] = 0
    Y = Y.astype('int64')
    f = np.bincount(Y)
    print(f/np.sum(f))
    return Y

In [20]:
def get_shift(data, shift_offset=2):
    g = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
    for i in range(1, shift_offset+1):
        data['gbp_gradient_p_'+str(i)] = shift(g, i, cval=0)

In [21]:
data = pd.read_csv('newstitle_gbp(uk-news,world,business,technology,money_2000-2016).csv')
data['gbp_gradient'] = np.gradient(data.as_matrix(columns=['gbp']).reshape(-1))
get_shift(data, shift_offset=2)

In [22]:
data.head()

,timestamp,title,gbp,gbp_gradient,gbp_gradient_p_1,gbp_gradient_p_2
0,946656000,"Big Macs, small horizons Y2K force outstrips s...",1.615700,-0.000700,0.000000,0.000000
1,946742400,Cock-fighting? Yes please Samaritans split ove...,1.615000,0.005512,-0.000700,0.000000
2,946828800,Hunters threaten Jospin with new battle of the...,1.626724,0.011001,0.005512,-0.000700
3,946915200,Leader: German sleaze inquiry Burundi peace in...,1.637002,0.005804,0.011001,0.005512
4,947001600,Women in record South Pole walk Deaths no coin...,1.638331,0.005379,0.005804,0.011001


In [23]:
Y = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
Y = shift(Y, -1, cval=3)
Y = binary(Y)
num_classes = 2

[ 0.47942553  0.52057447]


In [24]:
X_p = data.as_matrix(columns=['gbp_gradient_p_1', 'gbp_gradient_p_2'])
X_p[:,0] = binary(X_p[:,0])
X_p[:,1] = binary(X_p[:,1])
# X_p = data.as_matrix(columns=['gbp_gradient','gbp_gradient_p_1'])
# X_p[:,0] = binary(X_p[:,0])
# X_p[:,1] = binary(X_p[:,1])

[ 0.47974827  0.52025173]
[ 0.47990963  0.52009037]


In [25]:
feature_name = 'title'
vectorizer = TfidfVectorizer(min_df=50, ngram_range=(1, 4))
X = vectorizer.fit_transform(data[feature_name].tolist())
X

<6197x2730 sparse matrix of type '<class 'numpy.float64'>'
	with 476823 stored elements in Compressed Sparse Row format>

In [26]:
with_news = True

In [27]:
if with_news:
    X = hstack([X, X.power(2), X.power(3), X.power(4), X_p])
else:
    X = X_p

In [28]:
def create_model():
    model = Sequential()
    model.add(Dense(10, activation='tanh', input_shape=(X.shape[1],), bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='RMSprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    return model

In [29]:
# output = pd.DataFrame()
# output['timestamp'] = data['timestamp']
# output['gbp'] = data['gbp']
# output['gbp_gradient'] = data['gbp_gradient']
# output['gbp_gradient_binary'] = Y

In [30]:
def show_result(X_test, y_test, binary_pred, test_idx):
    target_names = ['decline','up']
    pred = np.argmax(model.predict(X_test), axis=1)
    binary_pred[test_idx] = pred
    #print(confusion_matrix(y_test, pred, labels=[0,1]))
    mat = confusion_matrix(y_test, pred, labels=[0,1])
    report = classification_report(y_test, pred, target_names=target_names) + '\n'
    return report

In [31]:
binary_pred = np.zeros(6197) #record pred value
skf = StratifiedKFold(n_splits=5, shuffle=True)
acc = []
num_epochs = 60 #37
history = ''
for i in range(1):
    for train_idx, test_idx in skf.split(X, Y):
        print ("Running Fold")
        model = create_model()
        if with_news:
            x = X.toarray()
        else:
            x = X
        y = np_utils.to_categorical(Y, num_classes)
        report = model.fit(x[train_idx], y[train_idx], 
                        epochs=num_epochs,
                        validation_data =(x[test_idx], y[test_idx]),
                        batch_size=7000,
                        verbose = 1)
        acc.append(model.evaluate(x[test_idx], y[test_idx], batch_size=2000)[1])
        history +=show_result(x[test_idx], Y[test_idx], binary_pred, test_idx)
    print(i)
print('acc:%.4f'%(np.mean(acc)))
print(history)

Running Fold
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 10)                109230    
_________________________________________________________________
dropout_21 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_22 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_23 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                110     

4956/4956 [==============================] - 0s - loss: 0.5945 - acc: 0.7123 - val_loss: 0.7245 - val_acc: 0.5125
Epoch 54/60
4956/4956 [==============================] - 0s - loss: 0.5889 - acc: 0.7272 - val_loss: 0.7264 - val_acc: 0.5222
Epoch 55/60
4956/4956 [==============================] - 0s - loss: 0.5807 - acc: 0.7296 - val_loss: 0.7283 - val_acc: 0.5165
Epoch 56/60
4956/4956 [==============================] - 0s - loss: 0.5832 - acc: 0.7260 - val_loss: 0.7302 - val_acc: 0.5222
Epoch 57/60
4956/4956 [==============================] - 0s - loss: 0.5817 - acc: 0.7238 - val_loss: 0.7327 - val_acc: 0.5133
Epoch 58/60
4956/4956 [==============================] - 0s - loss: 0.5745 - acc: 0.7359 - val_loss: 0.7342 - val_acc: 0.5189
Epoch 59/60
4956/4956 [==============================] - 0s - loss: 0.5727 - acc: 0.7389 - val_loss: 0.7363 - val_acc: 0.5197
Epoch 60/60
1241/1241 [==============================] - 0s
Running Fold
_________________________________________________________

4958/4958 [==============================] - 0s - loss: 0.6562 - acc: 0.6263 - val_loss: 0.6977 - val_acc: 0.4907
Epoch 46/60
4958/4958 [==============================] - 0s - loss: 0.6505 - acc: 0.6277 - val_loss: 0.6981 - val_acc: 0.4980
Epoch 47/60
4958/4958 [==============================] - 0s - loss: 0.6540 - acc: 0.6277 - val_loss: 0.6984 - val_acc: 0.4956
Epoch 48/60
4958/4958 [==============================] - 0s - loss: 0.6510 - acc: 0.6319 - val_loss: 0.6989 - val_acc: 0.4996
Epoch 49/60
4958/4958 [==============================] - 0s - loss: 0.6499 - acc: 0.6315 - val_loss: 0.6994 - val_acc: 0.4956
Epoch 50/60
4958/4958 [==============================] - 0s - loss: 0.6451 - acc: 0.6501 - val_loss: 0.6999 - val_acc: 0.4923
Epoch 51/60
4958/4958 [==============================] - 0s - loss: 0.6460 - acc: 0.6341 - val_loss: 0.7005 - val_acc: 0.4956
Epoch 52/60
4958/4958 [==============================] - 0s - loss: 0.6446 - acc: 0.6428 - val_loss: 0.7012 - val_acc: 0.4931
Epoc

4958/4958 [==============================] - 0s - loss: 0.6698 - acc: 0.5871 - val_loss: 0.6938 - val_acc: 0.5125
Epoch 38/60
4958/4958 [==============================] - 0s - loss: 0.6677 - acc: 0.5954 - val_loss: 0.6941 - val_acc: 0.5157
Epoch 39/60
4958/4958 [==============================] - 0s - loss: 0.6672 - acc: 0.5926 - val_loss: 0.6943 - val_acc: 0.5125
Epoch 40/60
4958/4958 [==============================] - 0s - loss: 0.6651 - acc: 0.5948 - val_loss: 0.6944 - val_acc: 0.5028
Epoch 41/60
4958/4958 [==============================] - 0s - loss: 0.6637 - acc: 0.6067 - val_loss: 0.6945 - val_acc: 0.5020
Epoch 42/60
4958/4958 [==============================] - 0s - loss: 0.6653 - acc: 0.6045 - val_loss: 0.6947 - val_acc: 0.5052
Epoch 43/60
4958/4958 [==============================] - 0s - loss: 0.6583 - acc: 0.6095 - val_loss: 0.6950 - val_acc: 0.5117
Epoch 44/60
4958/4958 [==============================] - 0s - loss: 0.6614 - acc: 0.5982 - val_loss: 0.6952 - val_acc: 0.4996
Epoc

4958/4958 [==============================] - 0s - loss: 0.6800 - acc: 0.5593 - val_loss: 0.6946 - val_acc: 0.5117
Epoch 30/60
4958/4958 [==============================] - 0s - loss: 0.6787 - acc: 0.5496 - val_loss: 0.6944 - val_acc: 0.5109
Epoch 31/60
4958/4958 [==============================] - 0s - loss: 0.6766 - acc: 0.5581 - val_loss: 0.6946 - val_acc: 0.5061
Epoch 32/60
4958/4958 [==============================] - 0s - loss: 0.6770 - acc: 0.5569 - val_loss: 0.6947 - val_acc: 0.5036
Epoch 33/60
4958/4958 [==============================] - 0s - loss: 0.6753 - acc: 0.5643 - val_loss: 0.6951 - val_acc: 0.5141
Epoch 34/60
4958/4958 [==============================] - 0s - loss: 0.6749 - acc: 0.5712 - val_loss: 0.6950 - val_acc: 0.5028
Epoch 35/60
4958/4958 [==============================] - 0s - loss: 0.6710 - acc: 0.5805 - val_loss: 0.6953 - val_acc: 0.5052
Epoch 36/60
4958/4958 [==============================] - 0s - loss: 0.6726 - acc: 0.5777 - val_loss: 0.6955 - val_acc: 0.4931
Epoc

4958/4958 [==============================] - 0s - loss: 0.6831 - acc: 0.5430 - val_loss: 0.6931 - val_acc: 0.5085
Epoch 22/60
4958/4958 [==============================] - 0s - loss: 0.6822 - acc: 0.5470 - val_loss: 0.6930 - val_acc: 0.5117
Epoch 23/60
4958/4958 [==============================] - 0s - loss: 0.6846 - acc: 0.5341 - val_loss: 0.6935 - val_acc: 0.5036
Epoch 24/60
4958/4958 [==============================] - 0s - loss: 0.6806 - acc: 0.5591 - val_loss: 0.6934 - val_acc: 0.5004
Epoch 25/60
4958/4958 [==============================] - 0s - loss: 0.6789 - acc: 0.5508 - val_loss: 0.6937 - val_acc: 0.5044
Epoch 26/60
4958/4958 [==============================] - 0s - loss: 0.6811 - acc: 0.5456 - val_loss: 0.6933 - val_acc: 0.5165
Epoch 27/60
4958/4958 [==============================] - 0s - loss: 0.6766 - acc: 0.5613 - val_loss: 0.6934 - val_acc: 0.5117
Epoch 28/60
4958/4958 [==============================] - 0s - loss: 0.6748 - acc: 0.5621 - val_loss: 0.6934 - val_acc: 0.5052
Epoc

In [32]:
# model = create_model()
# x = X.toarray()
# y = np_utils.to_categorical(Y, num_classes)
# model.fit(x, y, 
#         epochs=num_epochs,
#         batch_size=7000,
#         verbose = 1)

In [33]:
# model.save('model.h5')
# joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

In [34]:
# output['gbp_gradient_binary_pred'] = binary_pred
# output.to_pickle('output.pkl')